#### Training the Transformer model, as emphasized in "Attention is All You Need" implemented by Andrej Karpathy.

In [1]:
import torch
import tiktoken
import pickle
import argparse
from model import BigramLanguageModel
from dataclasses import dataclass


In [3]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1337)

In [4]:
@dataclass
class GPTConfig:
    block_size: int = 64
    batch_size:int = 256
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 2
    n_head: int = 2
    n_embd: int = 384
    dropout: float = 0.2
    learning_rate:float = 3e-4
    max_iters:int = 1000
    eval_iters:int = 384
    eval_interval:int = 250
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster


In [ ]:
config = GPTConfig()
model = BigramLanguageModel(config)
with open("model-gpt-01.pkl", 'rb') as f:
    model = pickle.load(f)

m =model.to(device)

In [ ]:
prompt = "Bad order"

In [ ]:
enc =tiktoken.get_encoding('gpt2')  
# generate from the model
context = torch.zeros(enc.encode_ordinary(prompt), dtype=torch.long, device=device)

In [ ]:
print(enc.decode(m.generate(context, max_new_tokens=1000)[0].tolist()))